In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [3]:
import numpy as np
import pandas as pd

df = pd.read_csv('./kaggle/input/spaceship-titanic/train.csv')
# df = pd.read_csv('./kaggle/input/spaceship-titanic/train_VIPNull_update.csv')
# df.isnull().sum()

# df_temp = df
# df['Cabin'] = df['Cabin'].str[0]
# df.insert(4, 'Cabin_num', df_temp['Cabin'].str[2:-2])
# df.insert(5, 'Cabin_port', df_temp['Cabin'].str[-1])
# df = pd.get_dummies(df, columns = ['Cabin'],dummy_na=False)
# df.isnull().sum()


In [4]:
# 범주형 데이터 변환
def df_replace(dataframe):
    
    global df_train,df
    df_temp = df
    df = dataframe
    df = df.replace({'55 Cancri e':0,'PSO J318.5-22':1,'TRAPPIST-1e':2})
    df = df.replace({'Earth':0,'Europa':1,'Mars':2})
    df = df.replace({False:0,True:1})

    # Cabin : 번호  
    df['Cabin'] = df['Cabin'].str[0]
    df.insert(4, 'Cabin_num', df_temp['Cabin'].str[2:-2])
    df.insert(5, 'Cabin_port', df_temp['Cabin'].str[-1])
    df = pd.get_dummies(df, columns = ['Cabin'], dummy_na=True)
    # df = df.replace({'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7})
    df = df.replace({'P':0,'S':1})

    col1 = df.columns[-9:].to_list()
    col2 = df.columns[3:-9].to_list()
    col3 = df.columns[:3].to_list()
    df_col = col3 + col1 + col2
    df = df[df_col]

    ## 범주형 데이터 확인할 수 있게 dict로 만들어 놓은 것
    # Cabin = {'A':0,'B':1,'C':2,'D':3,'E':4,'F':5,'G':6,'T':7}
    Cabin_port = {'P':0, 'S':1} #P : 배의 좌현(port side) S: 배의 우현(starboard side) 
    Destination = {'55 Cancri e': 0, 'PSO J318.5-22': 1,'TRAPPIST-1e':2}
    HomePlanet = {'Earth':0,'Europa':1,'Mars':2}
    CryoSleep = {'False':0,'True':1}
    VIP = {'False':0,'True':1}
    Transported = {'False':0,'True':1}


    df = df.astype({'HomePlanet':'float','CryoSleep':'float','Cabin_num':'float','Cabin_port':'float',
    'Destination':'float','Age':'float','VIP':'float','RoomService':'float','FoodCourt':'float'
    ,'ShoppingMall':'float','Spa':'float','VRDeck':'float','Transported':'float',
    'Cabin_A':'float','Cabin_B':'float','Cabin_C':'float','Cabin_D':'float','Cabin_E':'float','Cabin_F':'float',
    'Cabin_G':'float','Cabin_T':'float',
    })
    

df_replace(df)

df_train = df

In [5]:
# df_train[df_train['Cabin_A'].isnull()]
# df.isnull().sum()

### null값 처리
XGBOOST를 사용해서 pridct값을 null값으로 대체

In [6]:
df_train.columns

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin_A', 'Cabin_B',
       'Cabin_C', 'Cabin_D', 'Cabin_E', 'Cabin_F', 'Cabin_G', 'Cabin_T',
       'Cabin_nan', 'Cabin_num', 'Cabin_port', 'Destination', 'Age', 'VIP',
       'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck', 'Name',
       'Transported'],
      dtype='object')

In [18]:
df_train = df
df_train[['HomePlanet','Age']]

,HomePlanet,Age
0,1.0,39.0
1,0.0,24.0
2,1.0,58.0
3,1.0,33.0
4,0.0,16.0
...,...,...
8688,1.0,41.0
8689,0.0,18.0
8690,0.0,26.0
8691,1.0,32.0


In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
import pandas as pd
import seaborn as sns
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

def fillnullclassifier(column_name):

    global df_update

    df_column_name = df_train[df_train[column_name].isnull()]

    df_train01 = df_train.dropna(subset=[column_name])

    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)
    model = xgb.XGBClassifier(early_stopping_rounds=10)
    eval_set = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

    pred = model.predict(X_test)
    acc = accuracy_score(y_test,pred)
    print(acc)
    
    df_column_name_null = df_column_name.drop(column_name,axis=1)

    pred = model.predict(df_column_name_null.drop(['PassengerId','Name'],axis=1))

    df_column_name[column_name] = pred

    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnullclassifier('Destination')

0.6840869054609513


C:\Users\deepBlue\AppData\Local\Temp\ipykernel_10876\1768376489.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [22]:
df_update

,PassengerId,HomePlanet,CryoSleep,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,...,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8372,8956_02,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,2.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,Erina Bonnondry,1.0
8551,9130_01,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,41.0,0.0,0.0,0.0,0.0,0.0,0.0,Blers Corte,1.0
8616,9195_02,2.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,2.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,Ceros Purle,1.0
8621,9197_02,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,2.0,41.0,1.0,0.0,7964.0,0.0,3238.0,5839.0,Aludram Platch,0.0


In [102]:
df_update[df_update['Cabin_A'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin_A,Cabin_B,Cabin_C,Cabin_D,Cabin_E,Cabin_F,Cabin_G,...,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [103]:
df_update.to_csv('./kaggle/input/spaceship-titanic/train_Cabin_A_update.csv', index=False)